In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

****CODE START****

Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd 
import os
import random
from tqdm import tqdm
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from scipy.stats import mannwhitneyu

init_notebook_mode(connected=True) ## plotly init
seed = 123
random.seed = seed

In [ ]:
filenames_list = os.listdir('../input/Alcoholics/SMNI_CMI_TRAIN/Train/')

Data = pd.DataFrame({})

for file_names in tqdm(filenames_list):
    temp_df = pd.read_csv('../input/Alcoholics/SMNI_CMI_TRAIN/Train/' + file_names)
    Data = Data.append(temp_df)


In [ ]:
Data.head(100000)

**DATA STRUCTURE DETAILS**

In [ ]:
print(Data.shape)
print(Data.dtypes)

**DATA CLEAN CHECKING**

In [ ]:
for col in Data.columns:
    pct_missing = np.mean(Data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

**Dropping necessary columns **

In [ ]:
Data=Data.drop(['Unnamed: 0','sensor position','name'],axis = 1) 




In [ ]:
#Importing Data Visualization Libraries seaborn and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.violinplot(x="subject identifier",y="sensor value", hue="subject identifier", data=Data[:1000],color='green'); #Selecting only first 500 columns from Dataset
plt.show()

In [ ]:
ax= sns.stripplot(x="channel", y="matching condition", data=Data[:1000],color="red")

In [ ]:
g = sns.FacetGrid(data=Data[:1000],col='subject identifier')
g.map(plt.hist,"time",color="green")

In [ ]:
g = sns.FacetGrid(data=Data[:500],col='subject identifier',hue='matching condition')
g.map(plt.scatter,"sensor value","time",alpha=0.7,color="yellow")
g.add_legend();

# Plotting Pairwise Data Relationship

In [ ]:
Data_sample = Data[0:100]
#Data_sample = Data_sample.drop(['trial number'],axis=1)
Data_sample.head()

**These graph shows plots of all the selected attributes against each other**

In [ ]:
g = sns.PairGrid(Data_sample)
g.map(plt.scatter ,color="orange")

In [ ]:

g = sns.PairGrid(Data_sample, hue="subject identifier")
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()

#  Preprocessing of data

In [ ]:
#Importing LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#Data['sensor position'] = le.fit_transform(Data['sensor position'])
Data['subject identifier'] = le.fit_transform(Data['subject identifier'])
Data['matching condition'] = le.fit_transform(Data['matching condition'])
#Data['name'] = le.fit_transform(Data['name'])

In [ ]:
Data.head(100)

# Selection of data and target

In [ ]:
x= Data.iloc[:, Data.columns != 'subject identifier']
y= Data.iloc[:, 3]
# In the above code x represent data and y represent target

In [ ]:
#import the necessary module to train-test split

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
print("Shape of data training set {}".format(x_train.shape))    
print("Shape of data test set {}".format(x_test.shape)) 
print("Shape of target training set {}".format(y_train.shape)) 
print("Shape of target test set {}".format(y_train.shape)) 


# Various machine learning algorithms

1. **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
logmodel=LogisticRegression()
logmodel.fit(x_train,y_train)
y_pred=logmodel.predict(x_test)
print(y_pred)



#print the accuracy score
print("Logistic Regression Accuracy",accuracy_score(y_test,y_pred,normalize = True))


#code for confusion matrix
confusion_matrix(y_test,y_pred)

#from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(logmodel, classes=['0','1'])
visualizer.fit(x_train, y_train) # Fit the training data to the visualizer
visualizer.score(x_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data




**2.K-Nearest Neighbors**


K nearest neighbors is a simple algorithm that stores all available cases and classifies new cases based on a similarity measure 

In [ ]:
#import the necessary module
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

logmodel=KNeighborsClassifier()
logmodel.fit(x_train,y_train)
y_pred=logmodel.predict(x_test)
print(y_pred)




#print the accuracy score
print("K-Nearest Neighbors Accuracy",accuracy_score(y_test,y_pred,normalize = True))


#code for confusion matrix
confusion_matrix(y_test,y_pred)

from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(logmodel, classes=['0','1'])
visualizer.fit(x_train, y_train) # Fit the training data to the visualizer
visualizer.score(x_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data





**3.Naive Bayes**

Naive-Bayes algorithm calculates the probability of the connection of a feature with a target variable and then it selects the feature with the highest probability.

In [ ]:
#import the necessary module
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#Create an object of GaussianNB
gnb = GaussianNB()
#train the algorithm on taining data and predict using the testing data
#Fitting the data
gnb.fit(x_train,y_train)
#prediction 
y_pred = gnb.predict(x_test)
print(y_pred)

#print the accuracy score
print("Naive Bayes Accuracy ",accuracy_score(y_test,y_pred,normalize = True))

#Confusion Matrix
print(confusion_matrix(y_test,y_pred))



from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=['0','1'])
visualizer.fit(x_train, y_train) # Fit the training data to the visualizer
visualizer.score(x_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data


**4.Decision Tree**

Decision tree algorithm falls under the category of supervised learning. They can be used to solve both regression and classification problems.

In [ ]:
#import the necessary module
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#Create an object of GaussianNB
dec_tree = DecisionTreeClassifier(criterion="entropy", max_depth=5)
#train the algorithm on taining data and predict using the testing data
#Fitting the data
dec_tree.fit(x_train,y_train)
#prediction 
pred = dec_tree.predict(x_test)
print(pred)





#print the accuracy score
print("Decision Tree Accuracy ",accuracy_score(y_test,pred,normalize = True))

#Confusion Matrix
print(confusion_matrix(y_test,pred))





from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(dec_tree, classes=['0','1'])
visualizer.fit(x_train, y_train) # Fit the training data to the visualizer
visualizer.score(x_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data




from sklearn import tree
tree.plot_tree(dec_tree)

**5.Random Forest**

Random forest, like its name implies, consists of a large number of individual decision trees that operate as an ensemble. Each individual tree in the random forest spits out a class prediction and the class with the most votes becomes our model’s prediction 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#Create an object of GaussianNB
random_forest = RandomForestClassifier(criterion="entropy", max_depth=3)
#train the algorithm on taining data and predict using the testing data
#Fitting the data
random_forest.fit(x_train,y_train)
#prediction 
pred = random_forest.predict(x_test)
print(pred)






#print the accuracy score
print("Random Forest Accuracy ",accuracy_score(y_test,pred,normalize = True))





#Confusion Matrix
print(confusion_matrix(y_test,pred))

from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(random_forest, classes=['0','1'])
visualizer.fit(x_train, y_train) # Fit the training data to the visualizer
visualizer.score(x_test, y_test) # Evaluate the model on the test data